In [57]:
import xtgeo
import resqpy
import numpy as np
import pandas as pd
import pyarrow as pa
from resqpy.model import Model, new_model
from resqpy.property.string_lookup import StringLookup
from resqpy.crs import Crs

from fmu.sumo.explorer import Explorer
sumo = Explorer("dev")
table = sumo.get_table_by_uuid("402adb50-68f3-2bdf-9bb2-d4ebff1c69f9")

In [58]:
tabledata = table.metadata["data"]
tabledata

{'tagname': 'eclipse',
 'is_prediction': True,
 'format': 'arrow',
 'name': 'summary',
 'stratigraphic': False,
 'vertical_domain': 'depth',
 'is_observation': False,
 'content': 'timeseries',
 'spec': {'size': 239604, 'columns': ['DATE', 'GWPT:WI']}}

Instantiate new model of table

In [59]:
model = Model(new_epc=True)
#model = new_model(epc_file="table.epc") ### Table doesn't use any array data (in resqpy), thus initializing hdf5 file is redundant

Add a string lookup table of the table data to the model

In [60]:
match tabledata['format']:
    case 'csv':
        df = pd.read_csv(table.blob)
    case 'arrow':
        df = pd.read_parquet(table.blob, engine="pyarrow")
df

,DATE,GWPT:WI
0,2018-01-01 00:00:00,0.0
1,2018-01-02 00:00:00,0.0
2,2018-01-05 00:00:00,0.0
3,2018-01-06 00:00:00,0.0
4,2018-01-07 06:15:45,0.0
...,...,...
1711,2020-06-20 12:00:00,0.0
1712,2020-06-27 00:00:00,0.0
1713,2020-06-28 00:00:00,0.0
1714,2020-06-30 00:00:00,0.0


In [61]:
stringlu = StringLookup(model, title="String Table Lookup")

# Load table into dictionary where key is row number (key in StringLookup has to be integer)
tabledict = { i:row for i,row in enumerate(df.values.tolist()) }
stringlu.load_from_dict(tabledict)

# Append whole fmu metadata dict to mesh
stringlu.append_extra_metadata(table.metadata)

In [62]:
print(stringlu.as_dict())

{0: [Timestamp('2018-01-01 00:00:00'), 0.0], 1: [Timestamp('2018-01-02 00:00:00'), 0.0], 2: [Timestamp('2018-01-05 00:00:00'), 0.0], 3: [Timestamp('2018-01-06 00:00:00'), 0.0], 4: [Timestamp('2018-01-07 06:15:45'), 0.0], 5: [Timestamp('2018-01-08 04:14:06'), 0.0], 6: [Timestamp('2018-01-09 00:00:00'), 0.0], 7: [Timestamp('2018-01-09 14:34:02'), 0.0], 8: [Timestamp('2018-01-11 00:35:59'), 0.0], 9: [Timestamp('2018-01-11 20:12:41'), 0.0], 10: [Timestamp('2018-01-12 04:35:36'), 0.0], 11: [Timestamp('2018-01-12 18:47:14'), 0.0], 12: [Timestamp('2018-01-13 07:26:30'), 0.0], 13: [Timestamp('2018-01-14 06:04:57'), 0.0], 14: [Timestamp('2018-01-14 06:20:18'), 0.0], 15: [Timestamp('2018-01-14 09:01:16'), 0.0], 16: [Timestamp('2018-01-14 20:11:40'), 0.0], 17: [Timestamp('2018-01-15 12:40:17'), 0.0], 18: [Timestamp('2018-01-15 12:42:19'), 0.0], 19: [Timestamp('2018-01-16 00:59:21'), 0.0], 20: [Timestamp('2018-01-16 11:52:47'), 0.0], 21: [Timestamp('2018-01-16 21:42:54'), 0.0], 22: [Timestamp('201

Output our current table model

In [63]:
# Write all metadata to epc file
stringlu.create_xml()
model.store_epc("table.epc")

Try to reopen a new model from the files we just saved

In [64]:
# Recreate the model from epc file
read_model = Model("table.epc")
read_model.parts()

['obj_StringTableLookup_e7bad9aa-1b08-11ee-bc62-00d49ee52a08.xml']

In [65]:
read_model.titles()

['String Table Lookup']

Check that stored table is intact

In [66]:
read_stringlu_uuid = read_model.uuid(obj_type="obj_StringTableLookup")

read_stringlu = StringLookup(read_model, uuid=read_stringlu_uuid)
print("Rows:", len(read_stringlu.as_dict()))
print("Sample row:", read_stringlu.get_string(0))
print("Sample row:", read_stringlu.get_string(10))

Rows: 1716
Sample row: [Timestamp('2018-01-01 00:00:00'), 0.0]
Sample row: [Timestamp('2018-01-12 04:35:36'), 0.0]


In [67]:
print("Rows:", len(stringlu.as_dict()))
print("Sample row:", stringlu.get_string(0))
print("Sample row:", stringlu.get_string(10))

Rows: 1716
Sample row: [Timestamp('2018-01-01 00:00:00'), 0.0]
Sample row: [Timestamp('2018-01-12 04:35:36'), 0.0]


Check that extra metadata is intact

In [68]:
read_stringlu.extra_metadata

{'masterdata': "{'smda': {'field': [{'identifier': 'DROGON', 'uuid': '00000000-0000-0000-0000-000000000000'}]}}",
 'file': "{'checksum_md5': 'd7581e707a32c43f1cd437a762a1a604', 'relative_path': 'summary--GWPT:WI--eclipse--p10--iter-0'}",
 'access': "{'asset': {'name': 'Drogon'}}",
 'data': "{'tagname': 'eclipse', 'is_prediction': True, 'format': 'arrow', 'name': 'summary', 'stratigraphic': False, 'vertical_domain': 'depth', 'is_observation': False, 'content': 'timeseries', 'spec': {'size': 239604, 'columns': ['DATE', 'GWPT:WI']}}",
 'fmu': "{'context': {'stage': 'iteration'}, 'iteration': {'name': 'iter-0'}, 'aggregation': {'operation': 'p10'}, 'case': {'name': 'drogon_ahm_future-2023-03-09', 'user': {'id': 'dbs'}}}"}

In [69]:
stringlu.extra_metadata

{'masterdata': "{'smda': {'field': [{'identifier': 'DROGON', 'uuid': '00000000-0000-0000-0000-000000000000'}]}}",
 'file': "{'checksum_md5': 'd7581e707a32c43f1cd437a762a1a604', 'relative_path': 'summary--GWPT:WI--eclipse--p10--iter-0'}",
 'access': "{'asset': {'name': 'Drogon'}}",
 'data': "{'tagname': 'eclipse', 'is_prediction': True, 'format': 'arrow', 'name': 'summary', 'stratigraphic': False, 'vertical_domain': 'depth', 'is_observation': False, 'content': 'timeseries', 'spec': {'size': 239604, 'columns': ['DATE', 'GWPT:WI']}}",
 'fmu': "{'context': {'stage': 'iteration'}, 'iteration': {'name': 'iter-0'}, 'aggregation': {'operation': 'p10'}, 'case': {'name': 'drogon_ahm_future-2023-03-09', 'user': {'id': 'dbs'}}}"}